In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("debtData.csv")
print(df)

     CustomerID  Age  EducationLevel  YearsEmployed  Income  CardDebt  \
0             1   41               2              6      19     0.124   
1             2   47               1             26     100     4.582   
2             3   33               2             10      57     6.111   
3             4   29               2              4      19     0.681   
4             5   47               1             31     253     9.308   
..          ...  ...             ...            ...     ...       ...   
845         846   27               1              5      26     0.548   
846         847   28               2              7      34     0.359   
847         848   25               4              0      18     2.802   
848         849   32               1             12      28     0.116   
849         850   52               1             16      64     1.866   

     OtherDebt  Defaulted  DebtIncomeRatio  
0        1.073        0.0              6.3  
1        8.218        0.0        

In [ ]:
#Select columns for clustering
features = ['Age', 'CustomerID', 'Age', 'EducationLevel', 'YearsEmployed', 'Income', 'CardDebt', 'OtherDebt', 'Defaulted', 'DebtIncomeRatio']
X = df[features]

In [ ]:
#Scaling the columns
scaler = StandardScaler()
X_scaled = scaler.fit_transorm(X)

In [ ]:
#Elbow Method to determine optimal number of clusters
inertia = []
k_range = range(1, 6)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(k_range, inertia, marker= 'o')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()